# Challenge: Promotions

In this challenge, you'll develop codes to parse and analyze data returned from another API on Zalando such as [Promos homme (Men's Promotions)
](https://www.zalando.fr/promo-homme/) or [Promos femme (Women's Promotions)](https://www.zalando.fr/promo-femme/). The workflow is almost the same as in the guided lesson but you'll work with different data.

## Obtaining the link

Wrote your codes in the cell below to obtain the data from the API endpoint you choose. A recap of the workflow:

1. Examine the webpages and choose one that you want to work with.

1. Use Google Chrome's DevTools to inspect the XHR network requests. Find out the API endpoint that serves data to the webpage.

1. Test the API endpoint in the browser to verify its data.

1. Change the page number offset of the API URL to test if it's working.

In [3]:
# your code here

url = 'https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset=84'

## Reading the data

In the next cell, use Python to obtain data from the API endpoint you chose in the previous step. Workflow:

1. Import libraries.

1. Define the initial API endpoint URL.

1. Make request to obtain data of the 1st page. Flatten the data and store it in an empty object variable.

1. Find out the total page count in the 1st page data.

1. Use a FOR loop to make requests for the additional pages from 2 to page count. Append the data of each additional page to the flatterned data object.

1. Print and review the data you obtained.

In [13]:
# your code here

import json
import requests
import pandas as pd
from pandas import json_normalize # Se ajustó la librería por el "future warning" que mostraba
from urllib.request import urlopen

In [21]:
response = urlopen(url).read().decode('utf-8')
results = json.loads(response)

flatten_data = json_normalize(results)
flattened_data1 = json_normalize(flatten_data.articles[0])
flattened_data1.head()

,sku,name,sizes,url_key,media,brand_name,is_premium,family_articles,flags,product_group,delivery_promises,price.original,price.promotional,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,amount,price.base_price
0,BO754H000-Q11,VERBIER LANDO BRIEFBAG - Mallette - black,[One Size],bogner-verbier-lando-briefbag-sac-ordinateur-b...,[{'path': 'spp-media-p1/524ff6f58a52336fb013fe...,Bogner,True,[],"[{'key': 'discountRate', 'value': '-20%', 'tra...",accessoires,[],"199,99 €","159,99 €",False,False,False,False,NaN,NaN
1,1FI42E019-Q11,JAIRUS TAPE TRACK PANTS - Pantalon de survêtem...,"[XS, S, M, L]",fila-jairus-tape-track-pants-pantalon-de-surve...,[{'path': 'spp-media-p1/020749e6dea337f880ab00...,Fila,False,[],"[{'key': 'discountRate', 'value': '-20%', 'tra...",clothing,[],"49,95 €","39,95 €",False,False,False,False,NaN,NaN
2,C0T22O01U-Q11,CREWNECK BERLIN - T-shirt imprimé - black,"[XS, S, M, L, XL, XXL]",champion-reverse-weave-crewneck-berlin-t-shirt...,[{'path': 'spp-media-p1/d1e2645fcbdd38248032ab...,Champion Reverse Weave,False,[],"[{'key': 'discountRate', 'value': '-25%', 'tra...",clothing,[],"44,95 €","33,85 €",False,False,False,False,NaN,NaN
3,MA322O00J-C11,BASIC V-NECK - T-shirt basique - grey,"[S, M, L, XL, XXL, 3XL]",marc-opolo-basic-v-neck-t-shirt-basique-grey-m...,[{'path': 'spp-media-p1/619949e0a2323035b9dddc...,Marc O'Polo,False,[],"[{'key': 'discountRate', 'value': '-15%', 'tra...",clothing,[],"19,95 €","16,95 €",False,False,False,False,NaN,NaN
4,JA222T0RH-K11,JJIALVIN JJJACKET - Veste en jean - blue denim,"[XS, S, M, XXL]",jack-and-jones-jjialvin-jjjacket-veste-en-jean...,[{'path': 'spp-media-p1/36001bfcaa8b49bdb5a1c3...,Jack & Jones,False,[],"[{'key': 'discountRate', 'value': '-10%', 'tra...",clothing,[],"59,99 €","53,99 €",False,False,False,False,NaN,NaN


In [85]:
total_pages=results['pagination']['page_count']

df=pd.DataFrame()
for i in range(3):
    k=84*i
    response = urlopen(f'https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset={k}')
    response = urlopen(url).read().decode('utf-8')
    results = json.loads(response)
    flattened_data = json_normalize(results)
    flattened_data1 = json_normalize(flattened_data.articles[0])
    flattened_data1=flattened_data1.set_index('sku')
    df = df.append(flattened_data1)

df

,name,sizes,url_key,media,brand_name,is_premium,family_articles,flags,product_group,delivery_promises,price.original,price.promotional,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,amount,price.base_price
sku,,,,,,,,,,,,,,,,,,
A0Q22O0C8-Q11,FILGREE CREW - T-shirt imprimé - jet black/opt...,"[XS, S, M, L, XL, XXL]",allsaints-filgree-crew-t-shirt-imprime-jet-bla...,[{'path': 'spp-media-p1/69adca499680301fbb8186...,AllSaints,False,[],"[{'key': 'discountRate', 'value': '-30%', 'tra...",clothing,[],"54,95 €","38,65 €",False,False,False,False,NaN,NaN
HAS32G000-S11,GENTLE FACE WASH - Nettoyant visage - -,[],hanz-de-fuko-gentle-face-wash-nettoyant-visage...,[{'path': 'spp-media-p1/09c12e8eda1330e3aab318...,Hanz De Fuko,False,[],"[{'key': 'discountRate', 'value': '-15%', 'tra...",beauty,[],"27,95 €","23,85 €",False,False,False,False,237 ml,"10,06 € / 100 ml"
PU122O0BZ-Q11,FERRARI RACE TEE - T-shirt imprimé - black,"[S, M, L, XL, XXL]",puma-t-shirt-imprime-black-pu122o0bz-q11,[{'path': 'spp-media-p1/29159a6629d83e3eba6f3f...,Puma,False,[],"[{'key': 'discountRate', 'value': '-10%', 'tra...",clothing,[],"59,95 €","53,95 €",False,False,False,False,NaN,NaN
SE622A0IP-K11,SLIM BYRON - Blazer - navy blazer,"[46, 48, 50, 52, 54, 56]",selected-homme-slim-byron-blazer-navy-blazer-s...,[{'path': 'spp-media-p1/8de1611252633114ad057c...,Selected Homme,False,[],"[{'key': 'discountRate', 'value': '-20%', 'tra...",clothing,[],"94,99 €","75,99 €",False,False,False,False,NaN,NaN
TI122O040-E11,T-shirt à manches longues - mustard yellow,"[S, M, L, XL, XXL]",timberland-t-shirt-a-manches-longues-mustard-y...,[{'path': 'spp-media-p1/3cb0447e24163c899c2ab6...,Timberland,False,[],"[{'key': 'discountRate', 'value': '-25%', 'tra...",clothing,[],"50,00 €","37,50 €",False,False,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
REZ34H00O-S11,CURVACEOUS RINGLET - Produit coiffant - -,[],redken-curvaceous-ringlet-produit-coiffant-rez...,[{'path': 'spp-media-p1/89aa200675fb3f1a90fce9...,Redken,False,[],"[{'key': 'discountRate', 'value': '-30%', 'tra...",beauty,[],"31,95 €","22,45 €",False,False,False,False,180 ml,"12,47 € / 100 ml"
GS122G0TU-K12,TRIPLE A STRAIGHT C - Jean droit - melfort den...,"[28x30, 28x32, 29x30, 29x32, 29x34, 30x30, 30x...",g-star-alto-high-straight-jean-droit-melfort-d...,[{'path': 'spp-media-p1/d5d35aac882446f5a7696d...,G-Star,False,[],"[{'key': 'discountRate', 'value': 'Jusqu’à -30...",clothing,[],"129,95 €","90,95 €",True,False,True,False,NaN,NaN
EC112M023-O11,ST.1 HYBRID LITE - Chaussures à lacets - amber,"[39, 40, 43, 45, 46, 47, 48]",ecco-st1-hybrid-lite-chaussures-a-lacets-ec112...,[{'path': 'spp-media-p1/f7d77fc99a0e3abaa6401c...,ECCO,False,[],"[{'key': 'discountRate', 'value': '-25%', 'tra...",shoe,[],"139,95 €","104,95 €",False,False,False,False,NaN,NaN


## Bonus

Extract the following information from the data:

* The trending brand.

* The product(s) with the highest discount.

* The sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices).

In [86]:
# your code here

# Number of goods by brands

df.brand_name.value_counts()

Timberland        9
Tommy Hilfiger    9
Matinique         6
Puma              6
Selected Homme    6
                 ..
Calvin Klein      3
Geox              3
SIKSILK           3
Cotton On         3
HUGO              3
Name: brand_name, Length: 67, dtype: int64

In [87]:
# Trending brand
df.brand_name.value_counts().index[0]

'Timberland'

In [88]:
# Convert the df from string into numbers
df['price.original'] = df['price.original'].str.extract('(\d*,\d*)')
df['price.promotional'] = df['price.promotional'].str.extract('(\d*,\d*)')

In [89]:
df['price.original'] = [x.replace(',', '.') for x in df['price.original']]
df['price.promotional'] = [x.replace(',', '.') for x in df['price.promotional']]

df['discount_amount'] = df['price.original'].astype(float)-df['price.promotional'].astype(float)

In [91]:
df.sort_values(by='discount_amount', ascending = False)

,name,sizes,url_key,media,brand_name,is_premium,family_articles,flags,product_group,delivery_promises,price.original,price.promotional,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,amount,price.base_price,discount_amount
sku,,,,,,,,,,,,,,,,,,,
FEJ12M00C-T11,Derbies - embroidered colors,"[40, 41, 42, 43, 45]",fertini-derbies-embroidered-colors-fej12m00c-t11,[{'path': 'spp-media-p1/1231e63b17d63daa97f707...,Fertini,False,[],"[{'key': 'discountRate', 'value': '-46%', 'tra...",shoe,[],259.00,139.00,False,False,False,False,NaN,NaN,120.0
FEJ12M00C-T11,Derbies - embroidered colors,"[40, 41, 42, 43, 45]",fertini-derbies-embroidered-colors-fej12m00c-t11,[{'path': 'spp-media-p1/1231e63b17d63daa97f707...,Fertini,False,[],"[{'key': 'discountRate', 'value': '-46%', 'tra...",shoe,[],259.00,139.00,False,False,False,False,NaN,NaN,120.0
FEJ12M00C-T11,Derbies - embroidered colors,"[40, 41, 42, 43, 45]",fertini-derbies-embroidered-colors-fej12m00c-t11,[{'path': 'spp-media-p1/1231e63b17d63daa97f707...,Fertini,False,[],"[{'key': 'discountRate', 'value': '-46%', 'tra...",shoe,[],259.00,139.00,False,False,False,False,NaN,NaN,120.0
TG022T04V-Q11,HUTCH - Veste en cuir - black,"[S, M, L, XL, XXL]",tigha-hutch-veste-en-cuir-black-tg022t04v-q11,[{'path': 'spp-media-p1/fbfb1a8e4fb641548f9225...,Tigha,False,[],"[{'key': 'discountRate', 'value': '-35%', 'tra...",clothing,[],339.95,220.95,False,False,False,False,NaN,NaN,119.0
TG022T04V-Q11,HUTCH - Veste en cuir - black,"[S, M, L, XL, XXL]",tigha-hutch-veste-en-cuir-black-tg022t04v-q11,[{'path': 'spp-media-p1/fbfb1a8e4fb641548f9225...,Tigha,False,[],"[{'key': 'discountRate', 'value': '-35%', 'tra...",clothing,[],339.95,220.95,False,False,False,False,NaN,NaN,119.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CA312M05W-K11,BILL - Chaussures à lacets - navy,"[40, 41, 42, 43, 44, 45, 46]",camper-bill-chaussures-a-lacets-navy-ca312m05w...,[{'path': 'spp-media-p1/5e220e7256de3040b5816e...,Camper,False,[],"[{'key': 'campaign', 'value': 'Prix Mini', 'tr...",shoe,[],135.00,135.00,False,False,False,False,NaN,NaN,0.0
V1922T014-Q11,RAYNAN JACKET - Veste légère - black,"[S, M, L]",volcom-raynan-update-veste-legere-v1922t014-q11,[{'path': 'spp-media-p1/e9d3498eb261358790c52a...,Volcom,False,[],"[{'key': 'campaign', 'value': 'Prix Mini', 'tr...",clothing,[],130.00,130.00,False,False,False,False,NaN,NaN,0.0
6CA52R00P-Q11,BUSINESS CLASSIC PINSTRIPE TIE - Cravate - black,[One Size],calvin-klein-business-classic-pinstripe-tie-cr...,[{'path': 'spp-media-p1/89f9ac8a86dc34e6a35518...,Calvin Klein,False,[],"[{'key': 'campaign', 'value': 'Prix Mini', 'tr...",accessoires,[],44.95,44.95,False,False,False,False,NaN,NaN,0.0


In [92]:
total_discount = df['discount_amount'].sum()
print(total_discount)

4365.56
